MC 방법으로 정책 학습 ($\epsilon$-greedy)
Q1.최적 정책이 다름: 책에서는 s=0에서 [0.025,0.025,0.925,0.025]
Q2.Q도 다름 

In [10]:
import gymnasium as gym
import numpy as np

env = gym.make('FrozenLake-v1', render_mode='ansi')
env.reset()
print(env.render())


SFFF
FHFH
FFFH
HFFG



In [5]:
def generate_episode(env, policy):
    episode = []
    state, info = env.reset()

    while True:
        action = np.random.choice([0,1,2,3],1,p=policy[state])[0]
        next_state, reward, terminated, truncated, info = env.step(action)
        episode.append([state, action, reward])
        state = next_state
        if terminated or truncated:
            episode.append([next_state,None,None])
            break
    return episode

pi = [[0.25,0.25,0.25,0.25] for _ in range(env.observation_space.n)]
e = generate_episode(env, pi)
print(e)

[[0, np.int64(0), 0.0], [4, np.int64(2), 0.0], [0, np.int64(2), 0.0], [1, np.int64(2), 0.0], [2, np.int64(1), 0.0], [6, np.int64(1), 0.0], [7, None, None]]


In [6]:
#greedy_select = lambda x: np.random.choice(np.flatnonzero(x == x.max()))
greedy_select = lambda x: np.random.choice(np.argwhere(x == np.max(x)).flatten())
a = greedy_select(np.array([0.1, 0.3, 0.2, 0.3]))
print(a)

1


In [11]:
def epsilon_greedy_policy(env, gamma=0.99, epsilon=0.1):
    policy =[[0.25,0.25,0.25,0.25] for _ in range(env.observation_space.n)]
    #policy = np.ones((env.observation_space.n, env.action_space.n)) / env.action_space.n
    Q = np.zeros((env.observation_space.n, env.action_space.n))
    returns = [[[] for _ in range(env.action_space.n)] for _ in range(env.observation_space.n)]

    for i in range(1000):
        episode = generate_episode(env, policy)
        G = 0 # return
        for t in range(len(episode)-2,-1,-1):

            # update Q
            s, a, r = episode[t]
            G = gamma * G + r
            returns[s][a].append(G)
            Q[s][a] = np.mean(returns[s][a])
            amax = greedy_select(Q[s])

            # epsilon-greedy policy
            for b in range(env.action_space.n):
                if b == amax: # greedy action
                    policy[s][b] = 1 - epsilon + (epsilon / env.action_space.n)
                else: # non-greedy action
                    policy[s][b] = epsilon / env.action_space.n
    return policy, Q

pi, Q = epsilon_greedy_policy(env)
np.set_printoptions(precision=3)
print('optimal policy:\n',pi)
print('optimal Q:\n',Q)

env.close()


optimal policy:
 [[np.float64(0.025), np.float64(0.025), np.float64(0.025), np.float64(0.925)], [np.float64(0.025), np.float64(0.025), np.float64(0.025), np.float64(0.925)], [np.float64(0.925), np.float64(0.025), np.float64(0.025), np.float64(0.025)], [np.float64(0.025), np.float64(0.025), np.float64(0.025), np.float64(0.925)], [np.float64(0.925), np.float64(0.025), np.float64(0.025), np.float64(0.025)], [0.25, 0.25, 0.25, 0.25], [np.float64(0.925), np.float64(0.025), np.float64(0.025), np.float64(0.025)], [0.25, 0.25, 0.25, 0.25], [np.float64(0.025), np.float64(0.025), np.float64(0.025), np.float64(0.925)], [np.float64(0.025), np.float64(0.925), np.float64(0.025), np.float64(0.025)], [np.float64(0.025), np.float64(0.025), np.float64(0.925), np.float64(0.025)], [0.25, 0.25, 0.25, 0.25], [0.25, 0.25, 0.25, 0.25], [np.float64(0.025), np.float64(0.025), np.float64(0.925), np.float64(0.025)], [np.float64(0.025), np.float64(0.025), np.float64(0.925), np.float64(0.025)], [0.25, 0.25, 0.25, 0